In [93]:
import numpy as np
import pandas as pd
import matplotlib as mlt

#raw data from PDF via tabula
raw = pd.read_csv('tabula-death_penalty.csv')

#delete instances of multiple headers
#(there are a lot in the raw data)
df = raw[raw['SEX'] != "SEX"]

#print the data without repeated headers 
no_headers = df.to_csv('death_penalty_no_headers.csv')

#load in web database output:
#1967—present
edb = pd.read_csv('execution_database.csv')
espy = pd.read_csv('death_penalty_no_headers.csv')


In [94]:
#load in keys for in which states death 
#penalty is legal
legal = pd.read_csv('cleaners/legal.csv')

edb['Date'] = pd.to_datetime(edb['Date'])

#create new DF to cleanly concat two 
#datasets
edb_new = pd.DataFrame()

edb_new['NAME (Last First)'] = edb['Name']
edb_new['RACE'] = edb['Race']
edb_new['AGE'] = edb['Age']
edb_new['SEX'] = edb['Sex']
edb_new['CRIME'] = 'Missing'
edb_new['METHOD'] = edb['Method']
edb_new['DAY'] = edb['Date'].dt.day
edb_new['MONTH'] = edb['Date'].dt.month
edb_new['YEAR'] = edb['Date'].dt.year
edb_new['STATE'] = edb['State']
edb_new['DATE'] = edb['Date']

mask = (edb_new['DATE'] > '12/31/2002')

#make sure data from each for 2002 
#is the same so dfs don't overlap
#before concat

mask2002 = (edb_new['YEAR'] == 2002)
mask22002 = (espy['YEAR'] == 2002)

two002 = edb_new.loc[mask2002]
two2002 = espy.loc[mask22002]

In [95]:
edb_new = edb_new.loc[mask]

frames = [espy, edb_new]

result = pd.concat(frames)

result['executed'] = 'y'

result.to_csv('beforeclean.csv')

In [96]:
#fill in missing method values
result['METHOD'].replace(r'\s+','Missing',regex=True).replace('','Missing')

result['METHOD'].value_counts()

missingmethod = result.loc[result['METHOD'] == "Missing"]

#missingmethod['CRIME'].value_counts().to_csv('cleaningmethods.csv')

In [97]:
cleaner1 = pd.read_csv('cleaners/methodclean1.csv')

result = pd.merge(result, cleaner1, on="CRIME",how='outer')

result['METHOD'] = np.where(result['METHOD'] == 'Missing',result['NEWMETHOD'],result['METHOD'])

result['METHOD'].value_counts()

result = result.drop('NEWMETHOD',axis=1)

In [98]:
result['CRIME'].value_counts()

missingcrime = result.loc[result['CRIME'] == 'Missing']

missingcrime.to_csv('missingcrime.csv')

In [99]:
print 'Number of men and women executed in the 19th Century:' 

masknineteenth = (espy['YEAR'] >= 1800) & (espy['YEAR'] < 1900)

nineteenth = espy.loc[masknineteenth]

print nineteenth['SEX'].value_counts()
print nineteenth['RACE'].value_counts()

nineteenth.to_csv('nineteenth.csv')

Number of men and women executed in the 19th Century:
M    5078
F     175
Name: SEX, dtype: int64
Black                 2559
White                 2048
unknown                332
Native Amer.           253
Hispanic               115
Asian - Pacific Is      36
Name: RACE, dtype: int64


In [100]:
pivot = result.pivot_table(index=['STATE'],values='executed',aggfunc='count')

#new df from pivot table 
statecounts = pd.DataFrame(pivot)

statecounts.reset_index(inplace=True)

new = pd.merge(statecounts, legal, on='STATE')

print new.head(2)


  STATE  executed Death Penalty Legal? 
0    AK        12                    no
1    AL       757                   yes


In [101]:
pivot1 = new.pivot_table(index=['Death Penalty Legal? '],values='executed',aggfunc='sum')
print pivot1

Death Penalty Legal? 
m       1368
no      3343
yes    11136
Name: executed, dtype: int64


In [102]:
crimes = result['CRIME'].value_counts()
#crimes.to_csv('crimes.csv')

methods = result['METHOD'].value_counts()
#methods.to_csv('methods.csv')

cleanedcrimes = pd.read_csv('cleaners/cleaned_crimes.csv')
cleanedmethods = pd.read_csv('cleaners/methodcleaning.csv')

result = pd.merge(result, cleanedcrimes, on="CRIME", how='outer')
result = pd.merge(result, cleanedmethods, on="METHOD", how='outer')

result['METHOD CLEAN'] = np.where(result['METHOD CLEAN'] == "Missing",result['METHOD'],result['METHOD CLEAN'])



In [103]:
result['METHOD'] = result['METHOD CLEAN']

result = result.drop('METHOD CLEAN', axis=1)

result['METHOD'].value_counts()

Hanging             9290
Electrocution       4234
Lethal Injection     812
Gas                  541
Shot                 144
Other / Unknown       67
Burned                66
Break on Wheel        12
Hung in Chains         9
Gibbeted               6
Firing Squad           3
Bludgeoned             2
Pressing               1
Name: METHOD, dtype: int64

In [104]:
#most recent

mostrecentmethod = pd.DataFrame()

mostrecentmethod = result['METHOD'].value_counts()



Hanging             9290
Electrocution       4234
Lethal Injection     812
Gas                  541
Shot                 144
Other / Unknown       67
Burned                66
Break on Wheel        12
Hung in Chains         9
Gibbeted               6
Firing Squad           3
Bludgeoned             2
Pressing               1
Name: METHOD, dtype: int64


In [105]:
result['CRIME'] = result['CLEANED CRIME']

result = result.drop('CLEANED CRIME', axis=1)

result['CRIME'].value_counts()

Murder*                          12717
Rape                               946
Unknown / Other                    431
Slave Revolt                       258
Housebreaking - Burglary           249
Robbery                            158
Piracy                             130
Unspecified Felony                 107
Conspiracy to Murder               100
Arson                               97
Attempted Rape                      81
Attempted Murder                    60
Poisoning                           59
Horse Stealing                      51
Treason                             43
Rape - Robbery                      41
Desertion                           40
Accessory to Murder                 36
Witchcraft                          35
Counterfeiting                      31
Espionage                           27
Unknown                             25
Theft                               23
Forgery                             21
Aid Runaway Slave                   20
Burglary - Attempted Rape

In [106]:
#all clean!
result.to_csv('clean.csv')